# Práctica 4 - Inteligencia Artificial

### Grado Ingeniería Informática Tecnologías Informáticas - Curso 2019-20

### Técnicas metaheurísticas para optimización
### Procesos de Decisión de Markov

José Luis Ruíz Reina

In [1]:
import random

En esta práctica vamos a implementar algoritmos relacionados con Procesos de Decisión de Markov. 

Supondremos que un Procesos de Decisión de Markov (MDP, por sus siglas en inglés) va a ser un objeto de la siguiente clase (o mejor dicho, de una subclase de la siguiente clase). 

In [2]:
class MDP(object):

    """La clase genérica MDP tiene como métodos la función de recompensa R,
    la función A que da la lista de acciones aplicables a un estado, y la
    función T que implementa el modelo de transición. Para cada estado y
    acción aplicable al estado, la función T devuelve una lista de pares
    (ei,pi) que describe los posibles estados ei que se pueden obtener al
    plical la acción al estado, junto con la probabilidad pi de que esto
    ocurra. El constructor de la clase recibe la lista de estados posibles y
    el factor de descuento.

    En esta clase genérica, las funciones R, A y T aparecen sin definir. Un
    MDP concreto va a ser un objeto de una subclase de esta clase MDP, en la
    que se definirán de manera concreta estas tres funciones"""  

    def __init__(self,estados,descuento):

        self.estados=estados
        self.descuento=descuento

    def R(self,estado):
        pass

    def A(self,estado):
        pass
        
    def T(self,estado,accion):
        pass

Consideramos el siguiente problema:

A lo largo de su vida, una empresa pasa por situaciones muy distintas, que por simplificar resumiremos en que al inicio de cada campaña puede estar rica o pobre, y ser conocida o desconocida.  Para ello puede decidir en cada momento o bien invertir en publicidad, o bien optar por no hacer publicidad. Estas dos acciones no tienen siempre un resultado fijo, aunque podemos describirlo de manera probabilística:

* Si la empresa es rica y conocida y no invierte en publicidad, seguirá rica, pero existe un 50% de probabilidad de que se vuelva desconocida. Si gasta en publicidad, con toda seguridad seguirá conocida pero pasará a ser pobre.  
* Si la empresa es rica y desconocida y no gasta en publicidad, seguirá desconocida, y existe un 50% de que se vuelva pobre. Si gasta en publicidad, se volverá pobre, pero existe un 50% de probabilidades de que se vuelva conocida.
* Si la empresa es pobre y conocida y no invierte en publicidad, pasará a ser pobre y desconocida con un 50% de probabilidad, y rica y conocida en caso contrario. Si gasta en publicidad, con toda seguridad seguirá en la misma situación. 
* Si la empresa es pobre y desconocida, y no invierte en publicidad, seguirá en la misma situación con toda seguridad. Si gasta en publicidad, seguirá pobre, pero con un 50% de posibilidades pasará aser conocida. 

Supondremos que la recompensa en una campaña en la que la empresa es rica es de 10, y de 0 en en las que sea pobre. El objetivo es conseguir la mayor recompesa acumulada a lo largo del tiempo, aunque penalizaremos las ganancias obtenidas en campañas muy lejanas en el tiempo, introduciendo un factor de descuento de 0.9. 

### Ejercicio 1

Se pide representar el problema como un proceso de decisión de Markov, definiendo una clase Rica_y_Conocida, subclase de la clase MDP genérica, cuyo constructor recibe como entrada únicamente el factor de descuento, y en la que se definen de manera concreta los métodos R, A y T, según lo descrito. Para ello:
* La recompensa la guardaremos en un diccionario con claves "RC", "RD", "PC" y "PD", donde "R" es Rica, "P" es pobre, "C" es conocida y "D" es desconocida. Los valor del diccionario son las recompensas asociadas.
* Las acciones serán: "No publicidad" y "Gastar en publicidad"
* La transición también será un diccionario donde las claves son tuplas (Estado, acción) y los valores son listas de pares (Nuevo_estado, probabilidad). Por ejemplo un par clave-valor del diccionario será ("RC","No publicidad"):[("RC",0.5),("RD",0.5)]


In [3]:
# Solución:

class Rica_y_Conocida(MDP):
    
    def __init__(self,descuento=0.9):
        # RC: rica y conocida, RD: rica y desconocida, 
        # PC: pobre y conocida, PD: pobre y desconocida 
        self.Rdict={"RC":10,"RD":10,"PC":0,"PD":0}
        self.Tdict={("RC","No publicidad"):[("RC",0.5),("RD",0.5)],
                    ("RC","Gastar en publicidad"):[("PC",1)],
                    ("RD","No publicidad"):[("RD",0.5),("PD",0.5)],
                    ("RD","Gastar en publicidad"):[("PD",0.5),("PC",0.5)],
                    ("PC","No publicidad"):[("PD",0.5),("RC",0.5)],        
                    ("PC","Gastar en publicidad"):[("PC",1)],
                    ("PD","No publicidad"):[("PD",1)],
                    ("PD","Gastar en publicidad"):[("PD",0.5),("PC",0.5)]}
        super().__init__(["RC","RD","PC","PD"],descuento)
        
    def R(self,estado):
        return self.Rdict[estado]
        
    def A(self,estado):
        return ["No publicidad","Gastar en publicidad"]
        
    def T(self,estado,accion):
        return self.Tdict[(estado,accion)]

### Ejercicio 2

En general, dado un MDP, representaremos una política para el mismo como un diccionario cuyas claves son los estados, y los valores las acciones. Una política representa una manera concreta de decidir en cada estado la acción (de entre las aplicables a ese estado) que ha de aplicarse. 

Dado un MDP, un estado de partida, y una política concreta, podemos generar (muestrear) una secuencia de estados por los que iríamos pasando si vamos aplicando las acciones que nos indica la política: dado un estado de la secuencia, aplicamos a ese estado la acción que indique la política, y obtenemos un estado siguiente de manera aleatoria, pero siguiendo la distribución de probabilidad que indica el modelo de transición dado por el método T.  

Se pide definir una función "genera_secuencia_estados(mdp,pi,e,n)" que devuelva una secuencia de estados de longitud n, obtenida siguiendo el método anterior. Aquí mdp es objeto de la clase MDP, pi es una política, e un estado de partida y n la longitud de la secuencia. 

In [6]:
# Solución

# distr es un lista de pares (vi,pi) con los diferentes valores de la v.a. y
# sus probabilidades. 
def muestreo(distr):
    r=random.random()
    acum=0
    for v,p in distr:
        acum+=p
        if acum>r:
            return v

# Devuelve un valor y su probabilidad        
def muestreo_2(distr):
    r=random.random()
    acum=0
    for v,p in distr:
        acum+=p
        if acum>r:
            return v,p
        
        
def genera_secuencia_estados(mdp,pi,e,n):
    actual=e
    seq=[actual]
    for _ in range(n-1):
        actual=muestreo(mdp.T(actual,pi[actual]))
        seq.append(actual)
    return seq

# Devuelve la secuencia de estados y su probabilidad
def genera_secuencia_estados_2(mdp,pi,e,n):
    actual=e
    ac_prob = 1
    seq=[actual]
    for _ in range(n-1):
        actual,prob = muestreo(mdp.T(actual,pi[actual]))
        seq.append(actual)
        ac_prob *= prob
    return seq,ac_prob






Vemos a continuación algunos ejemplo de uso:

In [7]:
# Definimos una instancia de la subclase 
mdp_ryc=Rica_y_Conocida()

In [8]:
pi_ryc_ahorra={"RC":"No publicidad","RD":"No publicidad",
                    "PC":"No publicidad","PD":"No publicidad"}
genera_secuencia_estados(mdp_ryc,pi_ryc_ahorra,"PC",10)

# Posible resultado:
# ['PC', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 'PD']

['PC', 'RC', 'RC', 'RD', 'RD', 'RD', 'PD', 'PD', 'PD', 'PD']

In [9]:
pi_ryc_2={"RC":"No publicidad","RD":"Gastar en publicidad",
               "PC":"No publicidad","PD":"Gastar en publicidad"}
genera_secuencia_estados(mdp_ryc,pi_ryc_2,"RD",8)

# Posible resultado:
# ['RD', 'PC', 'RC', 'RC', 'RC', 'RC', 'RD', 'PC']

['RD', 'PD', 'PD', 'PC', 'PD', 'PC', 'RC', 'RC']

### Ejercicio 3

Dado un MDP y una secuencia de estados, valoramos dicha secuencia como la suma de las recompensas de los estados de la secuencias (cada una con el correspondiente descuento). Se pide definir una función valora_secuencia(seq,mdp) que calcula esta valoración.

In [10]:
# Solución

def valora_secuencia(seq,mdp):
    return sum(mdp.R(e)*(mdp.descuento**i) for i,e in enumerate(seq))
   

# def valora_secuencia(seq,mdp):
#     suma=0
#     i=0
#     for e in seq:
#         suma+=mdp.R(e)*(mdp.descuento**i)
#         i+=1
#     return suma


In [11]:
valora_secuencia(['PC', 'RC', 'RC', 'RC', 'RC', 'RC', 
                       'RD', 'RD', 'RD', 'PD', 'PD', 'PD', 
                       'PD', 'PD', 'PD', 'PD', 'PD', 'PD', 
                       'PD', 'PD'],mdp_ryc)

# Resultado:
# 51.2579511

51.2579511

In [12]:
valora_secuencia(['RD', 'PC', 'PD', 'PC', 'RC', 'RC', 
                        'RD', 'PD', 'PD', 'PC', 'RC', 'RC', 
                        'RC', 'RC', 'RC', 'RC'],mdp_ryc)

# Resultado:
# 44.11795212148159

44.11795212148159

### Ejercicio 4

Dada una política pi, la valoración de un estado e respecto de esa política, V^{pi}(e), se define como la media esperada de las valoraciones de las secuencias que se pueden generar teniendo dicho estado como estado de partida. Usando las funciones de los dos ejercicios anteriores, definir una función "estima_valor(e,pi,mdp,m,n)" que realiza una estimación aproximada del valor V^{pi}(e), para ello genera n secuencias de tamaño m, y calcula la media de sus valoraciones.  

In [13]:
# Solución:

# Creo que hay un error en esta solución. La media esperada es la suma de cada valoración de la secuencia
# multiplicada por su probabilidad y en esta solución se multiplica cada secuencia por 1/n

def estima_valor(e,pi,mdp,m,n):
    return (sum(valora_secuencia(genera_secuencia_estados(mdp,pi,e,m),mdp) 
                for _ in range(n)))/n

# Usando genera_secuencia_estados_2
def estima_valor_2(e,pi,mdp,m,n):
    suma = 0
    for _ in range(n):
        seq, prob = genera_secuencia_estdo_2(mdp,pi,e,m)
        val = valora_secuencia(seq)
        suma += val * prob
    return suma

In [14]:
estima_valor("PC",pi_ryc_ahorra,mdp_ryc,50,500)

# Respuesta posible:
# 14.531471247172597

13.593547270254689

In [15]:
estima_valor("PC",pi_ryc_2,mdp_ryc,50,500)

# Respuesta posible:
# 35.92126959549151

35.42294919352363

In [16]:
estima_valor("RC",pi_ryc_ahorra,mdp_ryc,60,700)

# Respuesta posible:
# 32.807558694112984

33.72271136420344

In [17]:
estima_valor("RC",pi_ryc_2,mdp_ryc,60,700)

# Respuesta posible:
# 50.09728516585913

50.84244245499118

### Ejercicio 5

Usando la función anterior, estimar la valoración de cada estado del problema "Rica y conocida", con las dos siguientes políticas:

* Aquella que sea cual sea el estado, siempre decide invertir en publicidad. 
* Aquella que sea cual sea el estado, siempre decide ahorrar. 

¿Cuál crees que es mejor? ¿Habrá alguna mejor que estas dos? ¿Cuál crees que sería la mejor política de todas? 

In [18]:
# Solución

pi_ryc_gastar={"RC":"Gastar en publicidad","RD":"Gastar en publicidad",
                  "PC":"Gastar en publicidad","PD":"Gastar en publicidad"}
pi_ryc_ahorra={"RC":"No publicidad","RD":"No publicidad",
                    "PC":"No publicidad","PD":"No publicidad"}

In [19]:
estima_valor("RC",pi_ryc_gastar,mdp_ryc,60,1000)

# Respuesta: 10.0

10.0

In [20]:
estima_valor("RC",pi_ryc_ahorra,mdp_ryc,60,1000)

# Respuesta: 33.354461818277635

32.99716281245544

In [21]:
estima_valor("RD",pi_ryc_gastar,mdp_ryc,60,1000)

# Respuesta: 10.0

10.0

In [22]:
estima_valor("RD",pi_ryc_ahorra,mdp_ryc,60,1000)

# Respuesta:18.17532275274187

18.403691026680985

In [23]:
estima_valor("PC",pi_ryc_gastar,mdp_ryc,60,1000)

# Respuesta: 0.0

0.0

In [24]:
estima_valor("PC",pi_ryc_ahorra,mdp_ryc,60,1000)

# Respuesta: estima_valor("PC",pi_ryc_ahorra,mdp_ryc,60,1000)

13.582350403662499

In [25]:
estima_valor("PD",pi_ryc_gastar,mdp_ryc,60,1000)

# Respuesta: 0.0

0.0

Se puede comprobar que salvo en el último caso, que la valoración es igual, las valoraciones que se consiguen con la segunda política son mayores que con la primera política. 

Ninguna de las dos políticas es la óptima, como se verá más adelante.

### Ejercicio 6

La función de valoración no se suele calcular mediante la técnica de muestreo vista en el ejercicio 4, sino como resultado de resolver un sistema de ecuaciones. Dicho sistema de ecuaciones se puede resolver de manera proximada de manera iterativa, tal como se explica en el tema.

Definir una función "valoración_respecto_política(pi,mdp, n)" que aplica dicho método iterativo (n iteraciones) para calcular la valoración V^{pi}. Dicha valoración debe devolverse como un diccionario que a cada estado e le asocia el valor "V^{pi}(e)" calculado.  

Aplicar la función para calcular la valoración asociada a las dos políticas que se dan en el ejercicio anterior, y comparara los valores obtenidos con los obtenidos mediante muestreo. 

In [ ]:
# Solución:

def valoración_respecto_política(pi,mdp, k):
    """Calcula una aproximación a la valoración de los estados respecto de la
    política pi, aplicando el método iterativo"""
    R, T, gamma = mdp.R, mdp.T, mdp.descuento
    V = {e:0 for e in mdp.estados}
    for _ in range(k):
        V1 = V.copy()
        for s in mdp.estados:
            V[s] = R(s) + gamma *(sum([p * V1[s1] for (s1,p) in T(s, pi[s])]))
    return V

Calculamos con esta función la valoración de las dos políticas anteriores.

In [ ]:
valoración_respecto_política(pi_ryc_gastar,mdp_ryc, 100)

# Resultado:
# {'RC': 10.0, 'RD': 10.0, 'PC': 0.0, 'PD': 0.0}

In [ ]:
valoración_respecto_política(pi_ryc_ahorra,mdp_ryc, 100)

# Resultado:
# {'RC': 33.05785123966942,
#  'RD': 18.18181818181818,
#  'PC': 14.876033057851238,
#  'PD': 0.0}

### Ejercicio 7

En el tema 3 se ha visto que la valoración de un estado se define como la mejor valoración que pueda tener el estado, respecto a todas las políticas posibles. Y la política óptima es aquella que en cada estado realiza la acción con la mejor valoración esperada (entendiendo por valoración esperada la suma de las valoraciones de los estados que podrían resultar al aplicar dicha acción, ponderadas por la probabilidad de que ocurra eso). De esta manera, la valoración de un estado es la valoración que la política óptima asigna al estado.

Para calcular tanto la valoración de los estados, como la política óptima, se han visto dos algoritmos: iteración de valores e iteración de políticas. En este ejercicio se pide implementar el algoritmo de iteración de políticas. En concreto, se pide definir una función "iteración_de_políticas(mdp,k)" que implementa el algoritmo de iteración de políticas, y devuelve dos diccionarios, uno con la valoración de los estados y otro con la política óptima. 

Comparar los resultados obtenidos con las políticas del ejercicio 5 y las valoraciones obtenidas.  

In [ ]:
# Solución:

def argmax(seq,f):
    max=float("-inf")
    amax=None
    for x in seq:
        fx=f(x)
        if fx>max:
            max=fx
            amax=x
    return amax

def valoración_esperada(acc,estado,V,mdp):
    """ Encuentra la valoración esperada de una acción respecto de una función
    de valoración V"""

    return sum((p * V[e] for (e,p) in mdp.T(estado, acc)))


def iteración_de_políticas(mdp,k):
    "Algoritmo de iteración de políticas"
    V = {e:0 for e in mdp.estados}
    pi = {e:random.choice(mdp.A(e)) for e in mdp.estados}
    while True:
        V = valoración_respecto_política(pi,mdp, k)
        actualizado = False
        for e in mdp.estados:
            acc = argmax(mdp.A(e), lambda a:valoración_esperada(a, e,V, mdp))
            if (acc != pi[e] and 
                valoración_esperada(acc, e,V, mdp) > valoración_esperada(pi[e], e,V, mdp)): # Por si hay empate
                pi[e] = acc
                actualizado = True
        if not actualizado:
            return pi,V

Calculamos la mejor política y su valoración con el MDP de Rica_y_Conocida. Como se ve, resulta que lo mejor es sólo gastar en publicidad cuando se es pobre y desconocido. Se observa que la valoración respecto de esa política es mejor, que las valoraciones con las política que se vieron en los ejercicios 5 y 6.   

In [ ]:
iteración_de_políticas(mdp_ryc,100)

# Respuesta
# ({'RC': 'No publicidad',
#   'RD': 'No publicidad',
#   'PC': 'No publicidad',
#   'PD': 'Gastar en publicidad'},
#  {'RC': 54.20053629623792,
#   'RD': 44.02311379672535,
#   'PC': 38.602953921506,
#   'PD': 31.584041852876634})